# 🚀 Étape 4 : Déploiement et API

## 📋 Objectifs de cette étape
1. **Charger** le meilleur modèle sauvegardé de l'étape 3
2. **Créer** une classe de service pour les prédictions
3. **Développer** une API FastAPI pour servir le modèle
4. **Implémenter** la validation des données d'entrée
5. **Tester** l'API avec différents exemples
6. **Créer** une interface web pour interagir avec l'API

---

## 🛠️ Configuration et Imports

In [ ]:
# Configuration générale
import warnings
warnings.filterwarnings('ignore')

# Manipulation des données
import pandas as pd
import numpy as np
from pathlib import Path
import json
import pickle
from datetime import datetime
from typing import Dict, List, Optional

# API et Web
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from fastapi.staticfiles import StaticFiles
from fastapi.responses import HTMLResponse
from pydantic import BaseModel, Field, validator
import uvicorn

# Machine Learning
from sklearn.preprocessing import RobustScaler

print("✅ Configuration terminée !")
print(f"📦 Pandas version: {pd.__version__}")
print(f"🚀 FastAPI prêt pour le déploiement")

## 📊 Chargement du Modèle et Métadonnées

Chargeons le meilleur modèle sauvegardé de l'étape 3.

In [ ]:
# 1. Chemins vers les fichiers
models_path = Path('../models')
processed_data_path = Path('../data/processed')

print("📂 CHARGEMENT DU MODÈLE ET MÉTADONNÉES")
print("=" * 45)

# 2. Rechercher les fichiers de modèle disponibles
model_files = list(models_path.glob('best_model_*.pkl'))
metadata_files = list(models_path.glob('model_metadata_*.json'))

if not model_files:
    print("❌ Aucun modèle trouvé !")
    print("💡 Assurez-vous d'avoir exécuté le notebook 03_modeling_evaluation.ipynb")
else:
    # Prendre le premier modèle trouvé
    model_file = model_files[0]
    metadata_file = metadata_files[0] if metadata_files else None
    
    print(f"📋 Fichiers trouvés:")
    print(f"   Modèle: {model_file.name}")
    if metadata_file:
        print(f"   Métadonnées: {metadata_file.name}")
    
    # 3. Charger le modèle
    with open(model_file, 'rb') as f:
        model = pickle.load(f)
    
    print(f"✅ Modèle chargé: {type(model).__name__}")
    
    # 4. Charger les métadonnées
    if metadata_file:
        with open(metadata_file, 'r') as f:
            model_metadata = json.load(f)
        
        print(f"✅ Métadonnées chargées:")
        print(f"   📊 R² Score: {model_metadata['performance']['test_r2']:.4f}")
        print(f"   📉 RMSE: {model_metadata['performance']['test_rmse']:,.0f}")
        print(f"   🔢 Features: {model_metadata['data_info']['features_count']}")
    
    # 5. Charger le scaler et les métadonnées de preprocessing
    preprocessing_metadata_path = processed_data_path / 'metadata.json'
    if preprocessing_metadata_path.exists():
        with open(preprocessing_metadata_path, 'r') as f:
            preprocessing_metadata = json.load(f)
        
        print(f"✅ Métadonnées de preprocessing chargées")
        print(f"   🔄 Scaler utilisé: {preprocessing_metadata['scaler_used']}")
    else:
        print("⚠️ Métadonnées de preprocessing non trouvées")
    
    print(f"\n🎯 Modèle prêt pour le déploiement !")

## 🏗️ Classe de Service de Prédiction

Créons une classe pour gérer les prédictions et la préparation des données.

In [ ]:
class HousePricePredictor:
    """Service de prédiction des prix immobiliers"""
    
    def __init__(self, model, feature_names, scaler_params=None):
        self.model = model
        self.feature_names = feature_names
        self.scaler = RobustScaler() if scaler_params else None
        
    def prepare_features(self, input_data: Dict) -> pd.DataFrame:
        """Prépare les features à partir des données d'entrée"""
        
        # Créer un DataFrame avec les features de base
        df = pd.DataFrame([input_data])
        
        # Engineering des features (reproduire les transformations de l'étape 2)
        df['price_per_sqft'] = 0  # Sera recalculé après prédiction si nécessaire
        df['rooms_total'] = df['bedrooms'] + df['bathrooms']
        df['area_per_room'] = df['area'] / df['rooms_total']
        df['bathroom_bedroom_ratio'] = df['bathrooms'] / df['bedrooms']
        
        # Variables d'équipements
        luxury_features = ['guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']
        df['luxury_score'] = df[luxury_features].sum(axis=1)
        df['has_luxury'] = (df['luxury_score'] > 0).astype(int)
        
        # Catégorisation des tailles
        def categorize_size(area):
            if area <= 5000:
                return 'small'
            elif area <= 8000:
                return 'medium'
            elif area <= 12000:
                return 'large'
            else:
                return 'very_large'
        
        df['size_category'] = df['area'].apply(categorize_size)
        
        # Variables d'interaction
        df['area_bedrooms_interaction'] = df['area'] * df['bedrooms']
        df['luxury_area_interaction'] = df['luxury_score'] * df['area']
        
        # One-hot encoding pour size_category
        size_dummies = pd.get_dummies(df['size_category'], prefix='size_category', drop_first=True)
        df = pd.concat([df, size_dummies], axis=1)
        
        # Sélectionner seulement les features numériques pour le modèle
        numeric_features = df.select_dtypes(include=[np.number]).columns.tolist()
        
        # Exclure price_per_sqft et size_category
        numeric_features = [f for f in numeric_features if f not in ['price_per_sqft', 'size_category']]
        
        # S'assurer que toutes les features attendues sont présentes
        for feature in self.feature_names:
            if feature not in df.columns:
                df[feature] = 0  # Valeur par défaut pour les features manquantes
        
        # Sélectionner les features dans le bon ordre
        df_features = df[self.feature_names]
        
        return df_features
    
    def predict(self, input_data: Dict) -> Dict:
        """Fait une prédiction de prix"""
        try:
            # Préparer les features
            features_df = self.prepare_features(input_data)
            
            # Faire la prédiction
            prediction = self.model.predict(features_df)[0]
            
            # Calculer le prix par pied carré
            price_per_sqft = prediction / input_data['area']
            
            return {
                'predicted_price': float(prediction),
                'price_per_sqft': float(price_per_sqft),
                'formatted_price': f"{prediction:,.0f}",
                'confidence': 'high' if hasattr(self.model, 'feature_importances_') else 'medium',
                'model_type': type(self.model).__name__
            }
            
        except Exception as e:
            raise ValueError(f"Erreur lors de la prédiction: {str(e)}")
    
    def get_feature_importance(self) -> Dict:
        """Retourne l'importance des features si disponible"""
        if hasattr(self.model, 'feature_importances_'):
            importance_dict = {}
            for feature, importance in zip(self.feature_names, self.model.feature_importances_):
                importance_dict[feature] = float(importance)
            
            # Trier par importance décroissante
            return dict(sorted(importance_dict.items(), key=lambda x: x[1], reverse=True))
        else:
            return {"message": "Feature importance not available for this model type"}

# Initialiser le service de prédiction
if 'model' in locals() and 'model_metadata' in locals():
    predictor = HousePricePredictor(
        model=model,
        feature_names=model_metadata['data_info']['feature_names']
    )
    print("✅ Service de prédiction initialisé !")
else:
    print("⚠️ Modèle non chargé, service de prédiction non disponible")

## 🔧 Modèles Pydantic pour la Validation

Définissons les modèles de données pour valider les entrées de l'API.

In [ ]:
# Modèles Pydantic pour la validation des données
class HouseFeatures(BaseModel):
    """Modèle pour les caractéristiques d'une maison"""
    
    # Caractéristiques de base
    area: float = Field(..., gt=0, description="Surface en pieds carrés")
    bedrooms: int = Field(..., ge=1, le=10, description="Nombre de chambres (1-10)")
    bathrooms: int = Field(..., ge=1, le=8, description="Nombre de salles de bain (1-8)")
    stories: int = Field(..., ge=1, le=5, description="Nombre d'étages (1-5)")
    
    # Équipements (0 ou 1)
    mainroad: int = Field(..., ge=0, le=1, description="Accès route principale (0/1)")
    guestroom: int = Field(..., ge=0, le=1, description="Chambre d'invités (0/1)")
    basement: int = Field(..., ge=0, le=1, description="Sous-sol (0/1)")
    hotwaterheating: int = Field(..., ge=0, le=1, description="Chauffage eau chaude (0/1)")
    airconditioning: int = Field(..., ge=0, le=1, description="Climatisation (0/1)")
    parking: int = Field(..., ge=0, le=10, description="Places de parking (0-10)")
    prefarea: int = Field(..., ge=0, le=1, description="Zone préférée (0/1)")
    
    # État du mobilier (0=non meublé, 1=semi-meublé, 2=meublé)
    furnishingstatus: int = Field(..., ge=0, le=2, description="État mobilier (0-2)")
    
    @validator('area')
    def validate_area(cls, v):
        if v < 1000 or v > 20000:
            raise ValueError('La surface doit être entre 1,000 et 20,000 pieds carrés')
        return v
    
    @validator('bathrooms', 'bedrooms')
    def validate_rooms(cls, v, field):
        if field.name == 'bathrooms' and v > 8:
            raise ValueError('Maximum 8 salles de bain')
        if field.name == 'bedrooms' and v > 10:
            raise ValueError('Maximum 10 chambres')
        return v
    
    class Config:
        schema_extra = {
            "example": {
                "area": 7420,
                "bedrooms": 4,
                "bathrooms": 1,
                "stories": 3,
                "mainroad": 1,
                "guestroom": 0,
                "basement": 0,
                "hotwaterheating": 0,
                "airconditioning": 1,
                "parking": 2,
                "prefarea": 1,
                "furnishingstatus": 1
            }
        }

class PredictionResponse(BaseModel):
    """Modèle pour la réponse de prédiction"""
    
    predicted_price: float = Field(..., description="Prix prédit")
    price_per_sqft: float = Field(..., description="Prix par pied carré")
    formatted_price: str = Field(..., description="Prix formaté")
    confidence: str = Field(..., description="Niveau de confiance")
    model_type: str = Field(..., description="Type de modèle utilisé")
    timestamp: str = Field(default_factory=lambda: datetime.now().isoformat())

class ErrorResponse(BaseModel):
    """Modèle pour les réponses d'erreur"""
    
    error: str = Field(..., description="Message d'erreur")
    details: Optional[str] = Field(None, description="Détails de l'erreur")
    timestamp: str = Field(default_factory=lambda: datetime.now().isoformat())

print("✅ Modèles Pydantic définis pour la validation !")

## 🚀 API FastAPI

Créons l'API FastAPI pour servir le modèle de prédiction.

In [ ]:
# Création de l'application FastAPI
app = FastAPI(
    title="🏠 House Price Predictor API",
    description="API de prédiction des prix immobiliers utilisant un modèle de Machine Learning",
    version="1.0.0",
    docs_url="/docs",
    redoc_url="/redoc"
)

# Configuration CORS pour permettre les requêtes depuis le frontend
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # En production, spécifier les domaines autorisés
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Variables globales pour le modèle
if 'predictor' not in locals():
    predictor = None
    model_info = None
else:
    model_info = model_metadata if 'model_metadata' in locals() else None

@app.get("/", response_class=HTMLResponse)
async def root():
    """Page d'accueil de l'API"""
    return """
    <html>
        <head>
            <title>🏠 House Price Predictor API</title>
            <style>
                body { font-family: Arial, sans-serif; margin: 40px; background: #f5f5f5; }
                .container { background: white; padding: 30px; border-radius: 10px; box-shadow: 0 2px 10px rgba(0,0,0,0.1); }
                h1 { color: #2c3e50; }
                .endpoint { background: #ecf0f1; padding: 10px; margin: 10px 0; border-radius: 5px; }
                .method { color: #27ae60; font-weight: bold; }
            </style>
        </head>
        <body>
            <div class="container">
                <h1>🏠 House Price Predictor API</h1>
                <p>API de prédiction des prix immobiliers utilisant un modèle de Machine Learning</p>
                
                <h2>📋 Endpoints disponibles:</h2>
                <div class="endpoint">
                    <span class="method">GET</span> <code>/</code> - Page d'accueil
                </div>
                <div class="endpoint">
                    <span class="method">GET</span> <code>/health</code> - État de l'API
                </div>
                <div class="endpoint">
                    <span class="method">GET</span> <code>/model/info</code> - Informations du modèle
                </div>
                <div class="endpoint">
                    <span class="method">POST</span> <code>/predict</code> - Prédiction de prix
                </div>
                <div class="endpoint">
                    <span class="method">GET</span> <code>/model/importance</code> - Importance des features
                </div>
                
                <h2>📚 Documentation:</h2>
                <p>
                    <a href="/docs" target="_blank">📖 Swagger UI Documentation</a><br>
                    <a href="/redoc" target="_blank">📋 ReDoc Documentation</a>
                </p>
                
                <h2>🧪 Interface de test:</h2>
                <p><a href="/static/index.html" target="_blank">🔬 Test Interface</a></p>
            </div>
        </body>
    </html>
    """

@app.get("/health")
async def health_check():
    """Vérification de l'état de l'API"""
    return {
        "status": "healthy",
        "model_loaded": predictor is not None,
        "timestamp": datetime.now().isoformat(),
        "version": "1.0.0"
    }

@app.get("/model/info")
async def get_model_info():
    """Informations sur le modèle chargé"""
    if predictor is None:
        raise HTTPException(status_code=503, detail="Modèle non chargé")
    
    return {
        "model_type": type(predictor.model).__name__,
        "features_count": len(predictor.feature_names),
        "performance": model_info.get('performance', {}) if model_info else {},
        "training_date": model_info.get('model_info', {}).get('training_date') if model_info else None,
        "features": predictor.feature_names[:10]  # Premiers 10 features
    }

@app.post("/predict", response_model=PredictionResponse, responses={400: {"model": ErrorResponse}})
async def predict_price(house_features: HouseFeatures):
    """Prédiction du prix d'une maison"""
    if predictor is None:
        raise HTTPException(status_code=503, detail="Modèle non chargé")
    
    try:
        # Convertir en dictionnaire
        input_data = house_features.dict()
        
        # Faire la prédiction
        result = predictor.predict(input_data)
        
        return PredictionResponse(**result)
        
    except ValueError as e:
        raise HTTPException(status_code=400, detail=str(e))
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Erreur interne: {str(e)}")

@app.get("/model/importance")
async def get_feature_importance():
    """Importance des features du modèle"""
    if predictor is None:
        raise HTTPException(status_code=503, detail="Modèle non chargé")
    
    return predictor.get_feature_importance()

@app.post("/predict/batch")
async def predict_batch(houses: List[HouseFeatures]):
    """Prédiction en lot pour plusieurs maisons"""
    if predictor is None:
        raise HTTPException(status_code=503, detail="Modèle non chargé")
    
    if len(houses) > 100:
        raise HTTPException(status_code=400, detail="Maximum 100 maisons par batch")
    
    results = []
    for i, house in enumerate(houses):
        try:
            input_data = house.dict()
            result = predictor.predict(input_data)
            result['house_index'] = i
            results.append(result)
        except Exception as e:
            results.append({"house_index": i, "error": str(e)})
    
    return {"predictions": results, "total": len(houses)}

print("✅ API FastAPI créée avec tous les endpoints !")
print("📋 Endpoints disponibles:")
print("   GET  /          - Page d'accueil")
print("   GET  /health    - État de l'API")
print("   GET  /model/info - Infos du modèle")
print("   POST /predict   - Prédiction simple")
print("   POST /predict/batch - Prédiction en lot")
print("   GET  /model/importance - Importance des features")

## 🧪 Test de l'API

Testons notre service de prédiction avec quelques exemples.

In [ ]:
# Test du service de prédiction
if 'predictor' in locals() and predictor is not None:
    print("🧪 TEST DU SERVICE DE PRÉDICTION")
    print("=" * 40)
    
    # Exemples de maisons à tester
    test_houses = [
        {
            "name": "Maison familiale standard",
            "data": {
                "area": 7420,
                "bedrooms": 4,
                "bathrooms": 1,
                "stories": 3,
                "mainroad": 1,
                "guestroom": 0,
                "basement": 0,
                "hotwaterheating": 0,
                "airconditioning": 1,
                "parking": 2,
                "prefarea": 1,
                "furnishingstatus": 1
            }
        },
        {
            "name": "Maison de luxe",
            "data": {
                "area": 12000,
                "bedrooms": 5,
                "bathrooms": 3,
                "stories": 2,
                "mainroad": 1,
                "guestroom": 1,
                "basement": 1,
                "hotwaterheating": 1,
                "airconditioning": 1,
                "parking": 4,
                "prefarea": 1,
                "furnishingstatus": 2
            }
        },
        {
            "name": "Petite maison économique",
            "data": {
                "area": 3500,
                "bedrooms": 2,
                "bathrooms": 1,
                "stories": 1,
                "mainroad": 1,
                "guestroom": 0,
                "basement": 0,
                "hotwaterheating": 0,
                "airconditioning": 0,
                "parking": 1,
                "prefarea": 0,
                "furnishingstatus": 0
            }
        }
    ]
    
    # Tester chaque maison
    for i, test_case in enumerate(test_houses, 1):
        print(f"\n🏠 Test {i}: {test_case['name']}")
        print("-" * 50)
        
        # Afficher les caractéristiques
        data = test_case['data']
        print(f"   📊 Surface: {data['area']:,} sq ft")
        print(f"   🛏️ Chambres: {data['bedrooms']} | 🛁 Salles de bain: {data['bathrooms']}")
        print(f"   🏢 Étages: {data['stories']} | 🚗 Parking: {data['parking']}")
        
        # Équipements
        amenities = []
        if data['guestroom']: amenities.append('Chambre invités')
        if data['basement']: amenities.append('Sous-sol')
        if data['hotwaterheating']: amenities.append('Chauffage')
        if data['airconditioning']: amenities.append('Climatisation')
        if data['prefarea']: amenities.append('Zone préférée')
        
        print(f"   🏆 Équipements: {', '.join(amenities) if amenities else 'Aucun équipement spécial'}")
        
        # Faire la prédiction
        try:
            result = predictor.predict(data)
            
            print(f"\n   🎯 RÉSULTAT DE LA PRÉDICTION:")
            print(f"   💰 Prix estimé: {result['formatted_price']}")
            print(f"   📊 Prix/sq ft: {result['price_per_sqft']:.2f}")
            print(f"   🤖 Modèle: {result['model_type']}")
            print(f"   ✅ Confiance: {result['confidence']}")
            
        except Exception as e:
            print(f"   ❌ Erreur: {str(e)}")
    
    print(f"\n✅ Tests terminés !")
    
else:
    print("⚠️ Service de prédiction non disponible - modèle non chargé")

## 📁 Création des Fichiers Statiques

Créons les fichiers pour l'interface web de test.

In [ ]:
# Créer le dossier static
static_path = Path('../static')
static_path.mkdir(exist_ok=True)

print("📁 CRÉATION DES FICHIERS STATIQUES")
print("=" * 40)

# Monter les fichiers statiques dans FastAPI
app.mount("/static", StaticFiles(directory=str(static_path)), name="static")

print(f"✅ Dossier static créé: {static_path}")
print("📂 Les fichiers HTML, CSS et JS seront créés dans ce dossier")
print("🌐 Accessibles via: http://localhost:8000/static/")

## 🚀 Lancement du Serveur

Instructions pour lancer l'API.

In [ ]:
# Fonction pour lancer le serveur
def start_server(host="127.0.0.1", port=8000):
    """Lance le serveur FastAPI"""
    print(f"🚀 LANCEMENT DU SERVEUR API")
    print("=" * 35)
    print(f"🌐 URL: http://{host}:{port}")
    print(f"📖 Documentation: http://{host}:{port}/docs")
    print(f"🧪 Interface de test: http://{host}:{port}/static/index.html")
    print("\n⚠️  Pour lancer le serveur, exécutez dans un terminal:")
    print(f"   uvicorn notebooks.04_deployment_api:app --host {host} --port {port} --reload")
    print("\n🛑 Pour arrêter: Ctrl+C")
    
    # Uncomment the line below to actually start the server in Jupyter
    # uvicorn.run(app, host=host, port=port, log_level="info")

# Afficher les instructions
start_server()

print(f"\n📋 RÉSUMÉ DE L'ÉTAPE 4:")
print("✅ Service de prédiction créé")
print("✅ API FastAPI implémentée")
print("✅ Validation des données avec Pydantic")
print("✅ Tests de l'API effectués")
print("✅ Endpoints pour prédictions simples et en lot")
print("✅ Documentation automatique générée")

print(f"\n🎯 PROCHAINES ÉTAPES:")
print("1. Créer les fichiers HTML, CSS et JS (voir cellules suivantes)")
print("2. Lancer le serveur avec la commande uvicorn")
print("3. Tester l'interface web")
print("4. Optionnel: Déployer en production (Docker, Heroku, etc.)")